# Q learning
## Brief
Suppose we have 5 rooms in a building connected by doors as shown in the figure below.  We'll number each room 0 through 4.  The outside of the building can be thought of as one big room (5).  Notice that doors 1 and 4 lead into the building from room 5 (outside). For this example, we'd like to put an agent in any room, and from that room, go outside the building (this will be our target room). In other words, the goal room is number 5. 
### Map
![map](map.jpg)
### Graph
![Graph](graph.jpg)
### Reference
[Reference](http://mnemstudio.org/path-finding-q-learning-tutorial.htm)
### Key algorithm
$$\Delta Q(s,a)=R(s,a)+\gamma\cdot\max_{\tilde{a}}{Q(\tilde{s},\tilde{a})}$$

$$Q(s,a) = (1-\alpha)Q(s,a)+\alpha\Delta Q(s,a)$$

## Import

In [1]:
import numpy as np
import random

## Initialize

In [2]:
Q=np.zeros([6,6])
R=np.array([[-1,-1,-1,-1,0,-1],
           [-1,-1,-1,0,-1,100],
           [-1,-1,-1,0,-1,-1],
           [-1,0,0,-1,0,-1],
           [0,-1,-1,0,-1,100],
            [-1,0,-1,-1,0,100]])
print("Q:\n{}\nR:\n{}".format(Q,R))

Q:
[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
R:
[[ -1  -1  -1  -1   0  -1]
 [ -1  -1  -1   0  -1 100]
 [ -1  -1  -1   0  -1  -1]
 [ -1   0   0  -1   0  -1]
 [  0  -1  -1   0  -1 100]
 [ -1   0  -1  -1   0 100]]


In [3]:
def train(R,targetState,Q=None,n_episode=20,learningRate=0.2,gamma=0.8,printInterval=None,verbose=False):
    rShape=np.shape(R)
    if rShape[0]!=rShape[1]:
        raise ValueError("The number of columns and rows in R didn't match. ")
    if Q==None:
        Q=np.zeros([rShape[0],rShape[1]])
    else:
        qShape=np.shape(Q)
        if qShape[0]!=rShape[0] or qShape[1]!=rShape[1]:
            raise ValueError("Size of Q and R didn't match. ")
    for episode in range(n_episode):
        state=random.randint(0,rShape[0]-1)
        step=0
        while state!=targetState:
            #Q(S,A) ← (1-α)*Q(S,A) + α*[R + γ*maxQ(S',a)] 
            step+=1
            actionSet=np.argwhere(R[state]>=0)
            action=actionSet[random.randint(0,np.shape(actionSet)[0]-1),0]
            if verbose and printInterval!=None and (episode+1)%printInterval==0:
                print("At episode {}\nState: {}\nAction: {}\nQ:\n{}".format(episode+1,state,action,Q))
            Q[state,action]=(1-learningRate)*Q[state,action]+learningRate*(R[state,action]+
                                                                           gamma*Q[action,np.argmax(Q[action])])
            state=action
        if printInterval!=None and (episode+1)%printInterval==0:
            print("At episode {}\nQ:\n{}".format(episode+1,Q))
    return Q

## Training #1

In [4]:
#Q(state, action) = R(state, action) + Gamma * Max[Q(next state, all actions)]
Q=train(R,targetState=5,n_episode=20,learningRate=1,gamma=0.8,printInterval=1,verbose=True)
print(Q)

At episode 1
State: 1
Action: 3
Q:
[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
At episode 1
State: 3
Action: 4
Q:
[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
At episode 1
State: 4
Action: 3
Q:
[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
At episode 1
State: 3
Action: 4
Q:
[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
At episode 1
State: 4
Action: 3
Q:
[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
At episode

## Online Inference

In [5]:
for state in range(5):
    print("Initial state:{}".format(state))
    while state!=5:
        #Didn't bother to validate the action
        actionSet=np.argwhere(Q[state]==np.max(Q[state]))
        action=actionSet[random.randint(0,np.shape(actionSet)[0]-1),0]
        print("Moving from room {} to room {}.".format(state,action))
        state=action

Initial state:0
Moving from room 0 to room 4.
Moving from room 4 to room 5.
Initial state:1
Moving from room 1 to room 5.
Initial state:2
Moving from room 2 to room 3.
Moving from room 3 to room 1.
Moving from room 1 to room 5.
Initial state:3
Moving from room 3 to room 4.
Moving from room 4 to room 5.
Initial state:4
Moving from room 4 to room 5.


## Training #2

In [6]:
#Q(S,A) ← (1-α)*Q(S,A) + α*[R + γ*maxQ(S',a)]
Q=train(R,n_episode=200,learningRate=0.3,gamma=0.85,targetState=5,printInterval=1,verbose=False)

At episode 1
Q:
[[  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  30.]
 [  0.   0.   0.   0.   0.   0.]]
At episode 2
Q:
[[  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  51.]
 [  0.   0.   0.   0.   0.   0.]]
At episode 3
Q:
[[  0.      0.      0.      0.      0.      0.   ]
 [  0.      0.      0.      0.      0.      0.   ]
 [  0.      0.      0.      0.      0.      0.   ]
 [  0.      0.      0.      0.     13.005   0.   ]
 [  0.      0.      0.      0.      0.     65.7  ]
 [  0.      0.      0.      0.      0.      0.   ]]
At episode 4
Q:
[[  0.         0.         0.         0.        16.7535     0.      ]
 [  0.         0.         0.         0.         0.        30.      ]
 [  0.         0.         0.         0.         0.         0.      ]
 [  0.         0.       

 [  0.           0.           0.           0.           0.           0.        ]]
At episode 27
Q:
[[  0.           0.           0.           0.          84.10730415   0.        ]
 [  0.           0.           0.          62.45847533   0.          91.76457   ]
 [  0.           0.           0.          57.23633049   0.           0.        ]
 [  0.          72.03158602  29.22768084   0.          83.79196579   0.        ]
 [ 69.33226284   0.           0.          66.30125647   0.          99.66767069]
 [  0.           0.           0.           0.           0.           0.        ]]
At episode 28
Q:
[[  0.           0.           0.           0.          84.29036893   0.        ]
 [  0.           0.           0.          62.45847533   0.          94.235199  ]
 [  0.           0.           0.          57.23633049   0.           0.        ]
 [  0.          73.82207557  29.22768084   0.          84.26399848   0.        ]
 [ 69.97994655   0.           0.          68.98136461   0.          99.66

 [  0.           0.           0.           0.           0.           0.        ]]
At episode 86
Q:
[[  0.           0.           0.           0.          84.99995548   0.        ]
 [  0.           0.           0.          72.24900803   0.          99.99774607]
 [  0.           0.           0.          72.24978566   0.           0.        ]
 [  0.          84.99529793  61.40482008   0.          84.9999718    0.        ]
 [ 72.24947519   0.           0.          72.24845609   0.          99.99998471]
 [  0.           0.           0.           0.           0.           0.        ]]
At episode 87
Q:
[[  0.           0.           0.           0.          84.99995548   0.        ]
 [  0.           0.           0.          72.24900803   0.          99.99842225]
 [  0.           0.           0.          72.24988275   0.           0.        ]
 [  0.          84.9961338   61.40708396   0.          84.9999718    0.        ]
 [ 72.24947519   0.           0.          72.24845609   0.          99.99

## Online inference

In [7]:
for state in range(5):
    print("Initial state:{}".format(state))
    while state!=5:
        #Didn't bother to validate the action
        actionSet=np.argwhere(Q[state]==np.max(Q[state]))
        action=actionSet[random.randint(0,np.shape(actionSet)[0]-1),0]
        print("Moving from room {} to room {}.".format(state,action))
        state=action

Initial state:0
Moving from room 0 to room 4.
Moving from room 4 to room 5.
Initial state:1
Moving from room 1 to room 5.
Initial state:2
Moving from room 2 to room 3.
Moving from room 3 to room 4.
Moving from room 4 to room 5.
Initial state:3
Moving from room 3 to room 4.
Moving from room 4 to room 5.
Initial state:4
Moving from room 4 to room 5.
